# Training & Generation for Paper "Responding to the Call: Exploring Automatic Music Composition Using a Knowledge-Enhanced Model"

In [2]:
# GPU settings
# On Mac (Apple Silicon), CUDA is not available.
# We will select device (CPU or MPS) later using torch, so no need to set CUDA_VISIBLE_DEVICES here.
device = torch.device("mps") 

In [1]:
import sys
import math
import time
import glob
import datetime
import random
import pickle
import json
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
from main_knowledge import *
import saver

In [3]:
# ###--- data ---###
path_data_root = './dataset/'
path_test_data = os.path.join(path_data_root, 'test.npz')
path_train_data = os.path.join(path_data_root, 'train.npz')
path_dictionary =  os.path.join(path_data_root, 'dictionary.pkl')

##uncomment the following to run on the complete training and test data
###--- data ---###
# path_data_root = '../train_test_data/'
# path_test_data = os.path.join(path_data_root, 'test.npz')
# path_train_data = os.path.join(path_data_root, 'train.npz')
# path_dictionary =  os.path.join(path_data_root, 'dictionary.pkl')



###--- training config ---###
path_exp = './exp'
batch_size =8
init_lr = 0.0001
max_grad_norm = 3
path_gendir = 'gen_midis'

import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")     # Apple GPU
else:
    device = torch.device("cpu")
# NaN が続く場合の対処: 以下で CPU に切り替えて再実行してみてください
# device = torch.device("cpu")

In [4]:
def get_train_data():
  dictionary = pickle.load(open(path_dictionary, 'rb'))
  event2word, word2event = dictionary
  train_data = np.load(path_train_data,allow_pickle=True)
  return train_data, event2word, word2event, dictionary
def get_test_data():
  dictionary = pickle.load(open(path_dictionary, 'rb'))
  event2word, word2event = dictionary
  test_data = np.load(path_test_data,allow_pickle=True)
  return test_data, event2word, word2event, dictionary


In [5]:
# Load data
train_data, event2word, word2event, dictionary = get_train_data()
test_data, event2word, word2event, dictionary = get_test_data()



# config
n_class = []
for key in event2word.keys():
    n_class.append(len(dictionary[0][key]))
print('num of classes:', n_class)

# # unpack
train_x = train_data['x']
train_y = train_data['y']
train_mask = train_data['mask']


fact_candidate=np.load('./dataset/knowledge.npy',allow_pickle=True)

# uncomment the following to load predefined external knowledge
# fact_candidate=torch.load('../train_test_data/external_knowledge/candidate_train')


test_x = test_data['x'][:]
test_y = test_data['y'][:]
test_mask = test_data['mask'][:]

# run
start_time = time.time()

num of classes: [234, 135, 18, 7, 130, 22, 130]


Initialize Model

In [6]:
net = TransformerModel(n_class)
info_load_model = False
# Use single-device training (CPU or MPS) on Mac
net.to(device)
net.train()
n_parameters = network_paras(net)
print('n_parameters: {:,}'.format(n_parameters))
# optimizers
optimizer = optim.Adam(net.parameters(), lr=init_lr)


>>>>>: [234, 135, 18, 7, 130, 22, 130]
n_parameters: 46,876,164


Train Model

In [8]:
saver_agent = saver.Saver(path_exp)
###TAsk1 ###
num_batch = len(train_x) // batch_size
candidate_number = 3
n_epoch = 5
start_time = time.time()
for epoch in range(n_epoch):
    acc_loss = 0
    acc_losses = np.zeros(7)
    n_updated = 0  # 有限 loss で更新したバッチ数（NaN スキップ用）
    with tqdm(range(num_batch)) as bar:
        for bidx in range(num_batch):  # num_batch
            # index
            bidx_st = batch_size * bidx
            bidx_ed = batch_size * (bidx + 1)
            # unpack batch data
            batch_x = train_x[bidx_st:bidx_ed]
            batch_y = train_y[bidx_st:bidx_ed]
            batch_mask = train_mask[bidx_st:bidx_ed]
            batch_x = torch.from_numpy(batch_x).long().to(device)
            batch_y = torch.from_numpy(batch_y).long().to(device)
            batch_mask = torch.from_numpy(batch_mask).float().to(device)

            # first task (loss_mask は float で渡す)
            zeros_token = torch.LongTensor([[0, 0, 0, 0, 0, 0, 0]]).to(device)
            zeros_mask_float = torch.zeros(1, 1, dtype=torch.float32, device=device)
            t1 = torch.cat([batch_y[:, :-1], zeros_token.expand(batch_y.shape[0], 1, 7)], 1)
            t2 = torch.cat([batch_y[:, 1:], zeros_token.expand(batch_y.shape[0], 1, 7)], 1)
            batch_mask1 = torch.cat([batch_mask[:, :-1], zeros_mask_float.expand(batch_y.shape[0], 1)], 1)

            src_mask = []
            for i in range(len(batch_x)):
                src_mask.append(int(torch.where(batch_x[i][:, 3] == 0)[0][0]))

            tgt_mask = []
            for i in range(len(t1)):
                tgt_mask.append(int(torch.where(t1[i][:, 3] == 0)[0][0]))

            losses = net.train_step(batch_x, t1, t2, src_mask, tgt_mask, batch_mask1, None)
            loss_task1 = (losses[0] + losses[1] + losses[2] + losses[3] + losses[4] + losses[5] + losses[6]) / 7

            # second task
            knowledge_base = {}
            knowledge_base['item'] = {}
            knw_mask_t = []
            batch_knowledge = {}
            for idx in range(candidate_number):
                batch_knowledge[idx] = fact_candidate[idx][bidx_st:bidx_ed]
                batch_knowledge[idx] = torch.from_numpy(batch_knowledge[idx]).long().to(device)

                knw_mask = []
                for j in range(len(batch_knowledge[idx])):
                    knw_mask.append(int(torch.where(batch_knowledge[idx][j][:, 3] == 0)[0][0]))
                knw_mask_t.append(knw_mask)

            for i in range(candidate_number):
                knowledge_base['item'][i] = net.forward_hidden(batch_knowledge[i], knw_mask_t[i])

            losses = net.train_step(batch_x, t1, t2, src_mask, tgt_mask, batch_mask1, knowledge_base['item'])
            loss_task2 = (losses[0] + losses[1] + losses[2] + losses[3] + losses[4] + losses[5] + losses[6]) / 7

            # third task
            loss_task3 = 0
            for can in range(candidate_number):
                t1 = torch.cat([batch_knowledge[can][:, :-1], zeros_token.expand(batch_knowledge[can].shape[0], 1, 7)], 1)
                t2 = torch.cat([batch_knowledge[can][:, 1:], zeros_token.expand(batch_knowledge[can].shape[0], 1, 7)], 1)
                mask_list = []
                for i in knw_mask_t[can]:
                    mask = np.concatenate([np.ones(i), np.zeros(256 - i)])
                    mask_list.append(mask)
                batch_mask = torch.tensor(mask_list, dtype=torch.float32, device=device)
                batch_mask1 = torch.cat([batch_mask[:, :-1], zeros_mask_float.expand(batch_y.shape[0], 1)], 1)
                tgt_mask = []
                for i in range(len(t1)):
                    tgt_mask.append(int(torch.where(t1[i][:, 3] == 0)[0][0]))
                losses = net.train_step(batch_x, t1, t2, src_mask, tgt_mask, batch_mask1, knowledge_base['item'])
                loss_3 = (losses[0] + losses[1] + losses[2] + losses[3] + losses[4] + losses[5] + losses[6]) / 7
                loss_task3 += loss_3

            loss_task3 = loss_task3 / candidate_number

            # final loss
            loss = loss_task1 + (loss_task2 + loss_task3) / 2

            # Update (NaN/Inf のときは更新をスキップしてモデルを守る)
            if torch.isfinite(loss):
                net.zero_grad()
                loss.backward()
                if max_grad_norm is not None:
                    clip_grad_norm_(net.parameters(), max_grad_norm)
                optimizer.step()
                acc_losses += np.array([l.item() for l in losses])
                acc_loss += loss.item()
                n_updated += 1
            else:
                net.zero_grad()

            # print
            sys.stdout.write('{}/{} | Loss: {:06f} | {:04f}, {:04f}, {:04f}, {:04f}, {:04f}, {:04f}, {:04f}\r'.format(
                bidx, num_batch, loss.item() if torch.isfinite(loss) else float('nan'),
                losses[0].item() if torch.isfinite(losses[0]) else float('nan'),
                losses[1].item() if torch.isfinite(losses[1]) else float('nan'),
                losses[2].item() if torch.isfinite(losses[2]) else float('nan'),
                losses[3].item() if torch.isfinite(losses[3]) else float('nan'),
                losses[4].item() if torch.isfinite(losses[4]) else float('nan'),
                losses[5].item() if torch.isfinite(losses[5]) else float('nan'),
                losses[6].item() if torch.isfinite(losses[6]) else float('nan')))
            sys.stdout.flush()
            bar.update()

    # epoch loss（有限で更新したバッチ数で割る）
    runtime = time.time() - start_time
    epoch_loss = acc_loss / n_updated if n_updated > 0 else float('nan')
    acc_losses = acc_losses / n_updated if n_updated > 0 else acc_losses
    print('------------------------------------')
    print('epoch: {}/{} | Loss: {} | time: {} | batches_ok: {}/{}'.format(
        epoch + 1, n_epoch, epoch_loss, str(datetime.timedelta(seconds=runtime)), n_updated, num_batch))

    # save model, with policy
    loss = epoch_loss
    if 0.4 < loss <= 1:
        fn = int(loss * 10) * 10
        saver_agent.save_model(net, name='loss_' + str(fn))
    elif 0.01 < loss <= 0.40:
        fn = int(loss * 100)
        saver_agent.save_model(net, name='loss_' + str(fn))
    elif loss <= 0.01:
        print('Finished')
    else:
        saver_agent.save_model(net, name='loss_high' + "_epoch_" + str(epoch))

  0%|          | 0/12 [00:00<?, ?it/s]

/opt/miniconda3/envs/callresponse/lib/python3.10/site-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/var/folders/1b/gvcxsgys679c56xf5qc8kq9h0000gn/T/ipykernel_5366/4207896577.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711403226260/work/torch/csrc/utils/tensor_new.cpp:278.)
  batch_mask = torch.tensor(mask_list, dtype=torch.float32, device=device)


  8%|▊         | 1/12 [00:59<10:58, 59.89s/it]

 17%|█▋        | 2/12 [04:28<24:35, 147.51s/it]

 25%|██▌       | 3/12 [08:24<28:08, 187.67s/it]

...

 75%|███████▌  | 9/12 [19:17<06:26, 128.74s/it]

 83%|████████▎ | 10/12 [21:16<04:11, 125.56s/it]

 92%|█████████▏| 11/12 [23:16<02:03, 123.93s/it]

100%|██████████| 12/12 [25:41<00:00, 128.48s/it]

------------------------------------
epoch: 5/5 | Loss: 2.3386694192886353 | time: 3:15:00.127150 | batches_ok: 12/12
 [*] saving model to ./exp, name: loss_high_epoch_4


In [7]:
device = torch.device("cpu")

net = TransformerModel(n_class, is_training=False)

path_saved_ckpt = "./exp/loss_high_epoch_4_params.pt"
state = torch.load(path_saved_ckpt, map_location="cpu")
net.load_state_dict(state)

net.to("cpu")
net.eval()

>>>>>: [234, 135, 18, 7, 130, 22, 130]


TransformerModel(
  (loss_func): CrossEntropyLoss()
  (word_emb_tempo): Embeddings(
    (lut): Embedding(234, 512)
  )
  (word_emb_chord): Embeddings(
    (lut): Embedding(135, 256)
  )
  (word_emb_barbeat): Embeddings(
    (lut): Embedding(18, 64)
  )
  (word_emb_type): Embeddings(
    (lut): Embedding(7, 32)
  )
  (word_emb_pitch): Embeddings(
    (lut): Embedding(130, 512)
  )
  (word_emb_duration): Embeddings(
    (lut): Embedding(22, 128)
  )
  (word_emb_velocity): Embeddings(
    (lut): Embedding(130, 512)
  )
  (pos_emb): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (in_linear): Linear(in_features=2016, out_features=512, bias=True)
  (linear_knowledge): Linear(in_features=1024, out_features=512, bias=True)
  (knowledge_selector): KnowledgeSelector(
    (linear): Linear(in_features=512, out_features=512, bias=True)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): Mul

In [8]:
batch_size=1
num_batch = len(test_x) // batch_size
output_total=[]
for bidx in range(5):
    bidx_st = batch_size * bidx
    bidx_ed = batch_size * (bidx + 1)
  # unpack batch data
    batch_x = test_x[bidx_st:bidx_ed]
    batch_y = test_x[bidx_st:bidx_ed]
    batch_mask = test_x[bidx_st:bidx_ed]
    batch_x = torch.from_numpy(batch_x).long().to(device)
    batch_y = torch.from_numpy(batch_y).long().to(device)
    batch_mask = torch.from_numpy(batch_mask).float().to(device)
    if isinstance(net, torch.nn.DataParallel):
          net = net.module

    src_mask=[]
    for i in range(len(batch_x)):
        src_mask.append(int(torch.where(batch_x[i][:,3]==0)[0][0]))
    output=net.inference(batch_x,src_mask,dictionary)
    output_total.append(output)
 

/opt/miniconda3/envs/callresponse/lib/python3.10/site-packages/torch/nn/modules/transformer.py:384: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711403226260/work/aten/src/ATen/NestedTensorImpl.cpp:179.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)


------ generate ------
bar: 1  ==Tempo_90        | D_o             | Beat_12         | Metrical        | 0               | 0               | Note_Velocity_39 | 
bar: 1  ==Tempo_102       | G_m             | 0               | SOC             | Note_Pitch_5    | Note_Duration_2040 | Note_Velocity_74 | 
bar: 1  ==0               | 0               | 0               | Note            | Note_Pitch_71   | Note_Duration_1320 | Note_Velocity_123 | 
bar: 1  ==0               | F#_m7           | 0               | Note            | 0               | Note_Duration_240 | Note_Velocity_89 | 
bar: 1  ==Tempo_91        | 0               | 0               | Note            | Note_Pitch_67   | Note_Duration_1920 | Note_Velocity_96 | 
bar: 1  ==CONTI           | B_7             | Beat_0          | Metrical        | 0               | 0               | Note_Velocity_58 | 
bar: 1  ==Tempo_76        | A#_/o7          | Beat_2          | SOC             | Note_Pitch_83   | Note_Duration_1920 | Note_Velocity_32

In [91]:
import os
os.makedirs("result", exist_ok=True)
for i in range(len(output_total)):
    write_midi(output_total[i],'./result/'+str(i)+'.mid',word2event)

TypeError: argument of type 'int' is not iterable

In [19]:
import os
os.makedirs("result", exist_ok=True)

call_0 = test_x[0]              # shape: (L, 7)
write_midi(call_0, "./result/call_0.mid", word2event)
print("saved -> ./result/call_0.mid")

saved -> ./result/call_0.mid
